## Convert tensorflow model to ONNX

In [21]:
import sys
 
# !{sys.executable} -m pip install --quiet --upgrade tensorflow==2.3.0
!{sys.executable} -m pip install --quiet --upgrade onnxruntime==1.4.0
!{sys.executable} -m pip install --quiet --upgrade onnxruntime-tools==1.4.0
# !{sys.executable} -m pip install --quiet --upgrade keras2onnx==1.7.0
# !{sys.executable} -m pip install --quiet transformers==3.0.2
# !{sys.executable} -m pip install --quiet onnxconverter_common
# !{sys.executable} -m pip install --quiet wget pandas

In [22]:
!{sys.executable} -m pip install --quiet --upgrade transformers
import transformers
print(transformers.__version__)
!python -m transformers.onnx --help

4.4.2
usage: Hugging Face ONNX Exporter tool [-h] -m MODEL
                                       [--feature {causal-lm,causal-lm-with-past,default,default-with-past,seq2seq-lm,seq2seq-lm-with-past,sequence-classification,sequence-classification-with-past}]
                                       [--opset OPSET] [--atol ATOL]
                                       output

positional arguments:
  output                Path indicating where to store generated ONNX model.

optional arguments:
  -h, --help            show this help message and exit
  -m MODEL, --model MODEL
                        Model's name of path on disk to load.
  --feature {causal-lm,causal-lm-with-past,default,default-with-past,seq2seq-lm,seq2seq-lm-with-past,sequence-classification,sequence-classification-with-past}
                        Export the model with some additional feature.
  --opset OPSET         ONNX opset version to export the model with (default
                        12).
  --atol ATOL           A

In [2]:
import tf2onnx; print(tf2onnx.__version__)
# import keras2onnx; print(keras2onnx.__version__)

In [3]:
# enable_overwrite = False
# total_runs = 100
# max_sequence_length = 512

# import os
# cache_dir = './cache_models'
# output_dir = './onnx_models'
# for directory in [cache_dir, output_dir]:
#     if not os.path.exists(directory):
#         os.makedirs(directory)
        
# import tensorflow as tf
# tf.config.set_visible_devices([], 'GPU') # Disable GPU for fair comparison

In [4]:
# from transformers import (TFBertForQuestionAnswering, BertTokenizer)

# #model_name_or_path = 'bert-large-uncased-whole-word-masking-finetuned-squad'
# model_name_or_path = "bert-base-cased"
# is_fine_tuned = (model_name_or_path == 'bert-large-uncased-whole-word-masking-finetuned-squad')

# # Load model and tokenizer
# tokenizer = BertTokenizer.from_pretrained(model_name_or_path, do_lower_case=True, cache_dir=cache_dir)
# model = TFBertForQuestionAnswering.from_pretrained(model_name_or_path, cache_dir=cache_dir)
# # Needed this to export onnx model with multiple inputs with TF 2.2
# model._saved_model_inputs_spec = None

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

Some layers of TFBertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# import numpy

# question, text = "What is Hyperplane?", "Hyperplane  is an end-to-end platform designed to take AI teams from ideation to production at breakthrough speeds. We built Hyperplane because we needed a powerful platform for our scientists to design, develop, deploy and maintain their own work in production."
# # Pad to max length is needed. Otherwise, position embedding might be truncated by constant folding.
# inputs = tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors='tf',
#                                max_length=max_sequence_length, pad_to_max_length=True, truncation=True)
# start_scores, end_scores = model(inputs)

# num_tokens = len(inputs["input_ids"][0])
# if is_fine_tuned:
#     all_tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
#     print("The answer is:", ' '.join(all_tokens[numpy.argmax(start_scores) : numpy.argmax(end_scores)+1]))

In [7]:
# import time
# start = time.time()
# for _ in range(total_runs):
#     start_scores, end_scores = model(inputs)
# end = time.time()
# print("Tensorflow Inference time for sequence length {} = {} ms".format(num_tokens, format((end - start) * 1000 / total_runs, '.2f')))

Tensorflow Inference time for sequence length 512 = 881.58 ms


## convert to onnx through transformers 

In [1]:
# !wget https://storage.googleapis.com/bert_models/2020_02_20/uncased_L-4_H-512_A-8.zip
# !unzip *.zip

In [33]:
%%time
# !export BERT_BASE_DIR="/root/ds-29faf59/starter_notebooks/trito/torch_model"

!transformers-cli convert --model_type bert \
  --tf_checkpoint ./torch_model/bert_model.ckpt \
  --config ./torch_model/bert_config.json \
  --pytorch_dump_output ./torch_model/pytorch_model.bin

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Building PyTorch model from configuration: BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

Converting TensorFlow checkpoint from /root/ds-29faf59/starter_notebooks/triton/torch_model/bert_model.ckp

In [2]:
!pip install -U transformers --quiet
!pip install -U tensorflow --quiet
!pip install -U torch --quiet


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
matrixprofile 1.1.10 requires protobuf==3.11.2, but you have protobuf 3.17.3 which is incompatible.
gql 3.0.0a6 requires graphql-core<3.2,>=3.1.5, but you have graphql-core 2.3.2 which is incompatible.


In [1]:
import tensorflow as tf
import torch
import transformers

print(f'tensorflow {tf.__version__}')
print(f'torch {torch.__version__}')
print(f'transformers {transformers.__version__}')

tensorflow 2.6.0
torch 1.7.1+cpu
transformers 4.4.2


In [2]:
from tensorflow.keras import activations, optimizers, losses
from transformers import BertTokenizer, BertForSequenceClassification

In [3]:
x = [
     'Great customer service! The food was delicious! Definitely a come again.',
     'The VEGAN options are super fire!!! And the plates come in big portions. Very pleased with this spot, I\'ll definitely be ordering again',
     'Come on, this place is family owned and operated, they are super friendly, the tacos are bomb.',
     'This is such a great restaurant. Multiple times during days that we don\'t want to cook, we\'ve done takeout here and it\'s been amazing. It\'s fast and delicious.',
     'Staff is really nice. Food is way better than average. Good cost benefit.',
     'pricing for this, while relatively inexpensive for a Las Vegas attraction, is completely over the top.',
     'At such a *fine* institution, I find the lack of knowledge and respect for the art appalling',
     'If I could give one star I would...I walked out before my food arrived the customer service was horrible!',
     'Wow the slowest drive thru I\'ve ever been at WOWWWW. Horrible I won\'t be coming back here ever again',
     'Service: 1 out of 5 stars. They will mess up your order, not have it ready after 30 mins calling them before. Worst ran family business Ive ever seen.'
]

y = [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]

In [26]:
# MODEL_PATH = "/root/ds-29faf59/wattpad_starters/train/imdb_bert"

# # MODEL_PATH = 'bert-base-uncased'
# MAX_LEN = 256

# review = x[0]
# tkzr = BertTokenizer.from_pretrained(MODEL_PATH)
# inputs = tkzr(review, max_length=MAX_LEN, truncation=True, padding=True)

# print(f'review: \'{review}\'')
# print(f'input ids: {inputs["input_ids"]}')
# print(f'attention mask: {inputs["attention_mask"]}')

In [27]:
# type(inputs["attention_mask"][0])

### may need to add  model_type key in the config

In [23]:
!python -m transformers.onnx --model="/root/ds-29faf59/wattpad_starters/train/imdb_bert" onnx_models/bert-finetuned/

file /root/ds-29faf59/wattpad_starters/train/imdb_bert/config.json not found
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/transformers/configuration_utils.py", line 524, in get_config_dict
    resolved_config_file = cached_path(
  File "/opt/conda/lib/python3.8/site-packages/transformers/file_utils.py", line 1419, in cached_path
    raise EnvironmentError(f"file {url_or_filename} not found")
OSError: file /root/ds-29faf59/wattpad_starters/train/imdb_bert/config.json not found

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.8/site-packages/transformers/onnx/__main__.py", line 71, in <module>
    main()
  File "/opt/conda/lib/python3.8/site-packages/t